<a href="https://colab.research.google.com/github/MoizAhmed2517/Deep_Learning_Projects/blob/main/Polyp_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Polyp Detection - Using YOLOv5

In [1]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et
import zipfile 
from google.colab import drive
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from shutil import move

In [2]:
os.chdir("/content/drive/MyDrive/Polyp Detection")

In [3]:
!pwd

/content/drive/MyDrive/Polyp Detection


In [4]:
!ls

 dataset   dataset.zip	'Polyp Detection.ipynb'   yolov5


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 14887, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 14887 (delta 0), reused 0 (delta 0), pack-reused 14882
Receiving objects: 100% (14887/14887), 13.86 MiB | 8.63 MiB/s, done.
Resolving deltas: 100% (10246/10246), done.


In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Polyp Detection/dataset.zip")
zip_ref.extractall()
zip_ref.close()

In [ ]:
len(os.listdir('/content/drive/MyDrive/Polyp Detection/dataset'))

2000

## Data Preparation - Reading XML Files

In [ ]:
xml_lst = glob("/content/drive/MyDrive/Polyp Detection/dataset/*.xml")

In [ ]:
xml_lst[:5]

['/content/drive/MyDrive/Polyp Detection/dataset/1.xml',
 '/content/drive/MyDrive/Polyp Detection/dataset/10.xml',
 '/content/drive/MyDrive/Polyp Detection/dataset/100.xml',
 '/content/drive/MyDrive/Polyp Detection/dataset/1000.xml',
 '/content/drive/MyDrive/Polyp Detection/dataset/101.xml']

In [ ]:
tree = et.parse(xml_lst[1])
root = tree.getroot()

In [ ]:
for child in root:
  print(child.tag, child.attrib)

folder {}
filename {}
path {}
source {}
size {}
object {}
object {}


In [ ]:
for object_xml in root.findall('object'):
  name = object_xml.find('name').text
  xmin = object_xml.find('bndbox/xmin').text
  ymin = object_xml.find('bndbox/ymin').text
  xmax = object_xml.find('bndbox/xmax').text
  ymax = object_xml.find('bndbox/ymax').text
  print(name, xmin, xmax, ymin, ymax)

polyp 270 362 247 391
polyp 45 359 209 446


In [ ]:
filename = root.find('filename').text
filename

'10.jpg'

In [ ]:
width = root.find('size/width').text
height = root.find('size/height').text

print(width, height)

611 530


### Making final helper function for parsing XML files & making their list and converting into dataframe for furthur use.

In [ ]:
def xml_parser(filename):
    tree = et.parse(filename)
    root = tree.getroot()
    filename = root.find('filename').text
    width = root.find('size/width').text
    height = root.find('size/height').text
    parser = []

    for object_xml in root.findall('object'):
      name = object_xml.find('name').text
      xmin = object_xml.find('bndbox/xmin').text
      ymin = object_xml.find('bndbox/ymin').text
      xmax = object_xml.find('bndbox/xmax').text
      ymax = object_xml.find('bndbox/ymax').text
      parser.append([filename, width, height, name, xmin, xmax, ymin, ymax])

    return parser

In [ ]:
xml_parser(xml_lst[1])

[['10.jpg', '611', '530', 'polyp', '270', '362', '247', '391'],
 ['10.jpg', '611', '530', 'polyp', '45', '359', '209', '446']]

In [ ]:
xml_parser(xml_lst[100])

[['189.jpg', '543', '528', 'polyp', '215', '315', '224', '328']]

In [ ]:
parsed_all_xml_files = list(map(xml_parser, xml_lst))

In [ ]:
data = reduce(lambda x,y : x+y, parsed_all_xml_files)
df = pd.DataFrame(data, columns=['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax'])

In [ ]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,1.jpg,619,529,polyp,243,427,19,142
1,1.jpg,619,529,polyp,187,256,224,347
2,1.jpg,619,529,polyp,345,412,159,267
3,1.jpg,619,529,polyp,229,321,303,416
4,1.jpg,619,529,polyp,340,382,407,459


In [ ]:
df.shape

(1460, 8)

In [ ]:
df['name'].value_counts()

polyp    1460
Name: name, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  1460 non-null   object
 1   width     1460 non-null   object
 2   height    1460 non-null   object
 3   name      1460 non-null   object
 4   xmin      1460 non-null   object
 5   xmax      1460 non-null   object
 6   ymin      1460 non-null   object
 7   ymax      1460 non-null   object
dtypes: object(8)
memory usage: 91.4+ KB


In [ ]:
df_updated = df.copy()
df_updated['width'] = df_updated['width'].astype("int64")
df_updated['height'] = df_updated['height'].astype("int64")
df_updated['xmin'] = df_updated['xmin'].astype("int64")
df_updated['xmax'] = df_updated['xmax'].astype("int64")
df_updated['ymin'] = df_updated['ymin'].astype("int64")
df_updated['ymax'] = df_updated['ymax'].astype("int64")
df_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  1460 non-null   object
 1   width     1460 non-null   int64 
 2   height    1460 non-null   int64 
 3   name      1460 non-null   object
 4   xmin      1460 non-null   int64 
 5   xmax      1460 non-null   int64 
 6   ymin      1460 non-null   int64 
 7   ymax      1460 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 91.4+ KB


In [ ]:
bbox_center_x = (df_updated['xmin'] + df_updated['xmax']) / 2
bbox_center_y = (df_updated['ymin'] + df_updated['ymax']) / 2
w = df_updated['xmax'] - df_updated['xmin']
h = (df_updated['ymax'] - df_updated['ymin'])
df_updated['center_x'] = np.round( bbox_center_x / df_updated['width'], 3)
df_updated['center_y'] = np.round( bbox_center_y / df_updated['height'], 3)
df_updated['width_norm'] = np.round( w / df_updated['width'], 3)
df_updated['height_norm'] = np.round( h / df_updated['height'], 3)

In [ ]:
#required labelling for YOLO -- Normalization of data as well
df_updated.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,width_norm,height_norm
0,1.jpg,619,529,polyp,243,427,19,142,0.541,0.152,0.297,0.233
1,1.jpg,619,529,polyp,187,256,224,347,0.358,0.540,0.111,0.233
2,1.jpg,619,529,polyp,345,412,159,267,0.611,0.403,0.108,0.204
3,1.jpg,619,529,polyp,229,321,303,416,0.444,0.680,0.149,0.214
4,1.jpg,619,529,polyp,340,382,407,459,0.583,0.819,0.068,0.098


In [ ]:
df_updated[df_updated['center_y'] < 0].sum()

filename       0.0
width          0.0
height         0.0
name           0.0
xmin           0.0
xmax           0.0
ymin           0.0
ymax           0.0
center_x       0.0
center_y       0.0
width_norm     0.0
height_norm    0.0
dtype: float64

In [ ]:
images = df['filename'].unique()
len(images)

997

In [ ]:
# Splitting in to 80%:20% percent ratios with random shuffle
img_df = pd.DataFrame(images, columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename'])
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename'])

In [ ]:
train_df = df_updated.query(f"filename in {img_train}")
test_df = df_updated.query(f"filename in {img_test}")

In [ ]:
len(train_df), len(test_df)

(1175, 285)

In [ ]:
train_df = train_df[train_df['name'] == 'polyp'].assign(id=0) 
train_df

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,width_norm,height_norm,id
0,1.jpg,619,529,polyp,243,427,19,142,0.541,0.152,0.297,0.233,0
1,1.jpg,619,529,polyp,187,256,224,347,0.358,0.540,0.111,0.233,0
2,1.jpg,619,529,polyp,345,412,159,267,0.611,0.403,0.108,0.204,0
3,1.jpg,619,529,polyp,229,321,303,416,0.444,0.680,0.149,0.214,0
4,1.jpg,619,529,polyp,340,382,407,459,0.583,0.819,0.068,0.098,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,994.jpg,621,531,polyp,488,607,268,399,0.882,0.628,0.192,0.247,0
1454,995.jpg,720,576,polyp,321,401,295,363,0.501,0.571,0.111,0.118,0
1457,997.jpg,600,528,polyp,287,391,304,427,0.565,0.692,0.173,0.233,0
1458,998.jpg,720,576,polyp,347,483,208,328,0.576,0.465,0.189,0.208,0


In [ ]:
test_df = test_df[test_df['name'] == 'polyp'].assign(id=0)
test_df

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,width_norm,height_norm,id
38,12.jpg,626,547,polyp,185,336,94,213,0.416,0.281,0.241,0.218,0
46,123.jpg,490,531,polyp,284,452,134,298,0.751,0.407,0.343,0.309,0
47,124.jpg,571,531,polyp,200,403,212,384,0.528,0.561,0.356,0.324,0
48,124.jpg,571,531,polyp,245,399,362,445,0.564,0.760,0.270,0.156,0
55,13.jpg,622,531,polyp,150,290,107,235,0.354,0.322,0.225,0.241,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,98.jpg,622,530,polyp,462,617,166,423,0.867,0.556,0.249,0.485,0
1426,981.jpg,720,576,polyp,218,434,96,347,0.453,0.385,0.300,0.436,0
1449,991.jpg,621,520,polyp,275,557,144,349,0.670,0.474,0.454,0.394,0
1455,996.jpg,585,530,polyp,294,423,4,199,0.613,0.192,0.221,0.368,0


In [ ]:
train_folder = '/content/drive/MyDrive/Polyp Detection/dataset/train'
test_folder = '/content/drive/MyDrive/Polyp Detection/dataset/test'

os.mkdir(train_folder)
os.mkdir(test_folder)

In [ ]:
cols = ['filename', 'id', 'center_x',	'center_y',	'width_norm',	'height_norm']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [ ]:
def save_data(filename, folder_path, group_obj):
  src = os.path.join('/content/drive/MyDrive/Polyp Detection/dataset', filename)
  dst = os.path.join(folder_path, filename)
  move(src, dst)

  text_filename = os.path.join(folder_path, os.path.splitext(filename)[0] + '.txt')
  group_obj.get_group(filename).set_index('filename').to_csv(text_filename, sep=' ', index=False, header=False)

In [ ]:
filename_series = pd.Series(groupby_obj_train.groups.keys())
filename_series.apply(save_data, args=(train_folder, groupby_obj_train))

0      None
1      None
2      None
3      None
4      None
       ... 
793    None
794    None
795    None
796    None
797    None
Length: 798, dtype: object

In [ ]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data, args=(test_folder, groupby_obj_test))

0      None
1      None
2      None
3      None
4      None
       ... 
194    None
195    None
196    None
197    None
198    None
Length: 199, dtype: object

In [ ]:
for i in os.listdir('/content/drive/MyDrive/Polyp Detection/dataset'):
  if os.path.splitext(i)[1] == '.xml':
    path = '/content/drive/MyDrive/Polyp Detection/dataset/' + i
    os.remove(path)

In [5]:
!pwd

/content/drive/MyDrive/Polyp Detection


In [6]:
os.chdir('/content/drive/MyDrive/Polyp Detection/yolov5') 

In [7]:
!ls

benchmarks.py	 export.py    README.zh-CN.md	tutorial.ipynb	yolov5x6.pt
CITATION.cff	 hubconf.py   requirements.txt	utils		yolov5x.pt
classify	 LICENSE      runs		val.py
CONTRIBUTING.md  models       segment		yolov5l6.pt
data		 __pycache__  setup.cfg		yolov5n.pt
detect.py	 README.md    train.py		yolov5s.pt


In [8]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 184 kB 14.8 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 1.6 MB 61.8 MB/s 


In [ ]:
!python train.py --data data.yaml --weights yolov5x.pt --batch 8 --name Model --img 640 --epochs 5

train: weights=yolov5x.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, i

In [ ]:
!python train.py --data data.yaml --weights yolov5x.pt --batch 8 --name Model --img 640 --epochs 15

train: weights=yolov5x.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=15, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, 

In [ ]:
!python train.py --data data.yaml --weights yolov5x.pt --batch 8 --name Model --img 640 --epochs 25

train: weights=yolov5x.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, 

In [ ]:
!python train.py --data data.yaml --weights yolov5x.pt --batch 8 --name Model --img 640 --epochs 35

train: weights=yolov5x.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=35, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, 

In [ ]:
!python train.py --data data.yaml --weights yolov5x.pt --batch 8 --name Model --img 640 --epochs 30

train: weights=yolov5x.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 8), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (21/21), done.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.1

In [ ]:
!python train.py --data data.yaml --weights yolov5x.pt --batch 8 --name Model --img 640 --epochs 32

train: weights=yolov5x.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=32, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bia

In [ ]:
!python train.py --data data.yaml --weights yolov5l6.pt --batch 4 --name Model --img 1280 --epochs 5

train: weights=yolov5l6.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bi

In [ ]:
!python train.py --data data.yaml --weights yolov5l6.pt --batch 4 --name Model --img 1280 --epochs 10

train: weights=yolov5l6.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_b

In [ ]:
!nvidia-smi

Tue Dec 27 19:51:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python train.py --data data.yaml --weights yolov5l6.pt --batch 4 --name Model --img 1280 --epochs 15

train: weights=yolov5l6.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=15, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_b

In [ ]:
!python train.py --data data.yaml --weights yolov5n.pt --batch 8 --name Model --img 640 --epochs 5

train: weights=yolov5n.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias

In [ ]:
!python train.py --data data.yaml --weights yolov5n.pt --batch 8 --name Model --img 640 --epochs 10

train: weights=yolov5n.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bia

In [ ]:
!python train.py --data data.yaml --weights yolov5n.pt --batch 8 --name Model --img 640 --epochs 35

train: weights=yolov5n.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=35, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bia

In [ ]:
!python train.py --data data.yaml --weights yolov5n.pt --batch 8 --name Model --img 640 --epochs 25

train: weights=yolov5n.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bia

In [ ]:
!python train.py --data data.yaml --weights yolov5n.pt --batch 8 --name Model --img 640 --epochs 15

train: weights=yolov5n.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=15, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bia

In [ ]:
!python train.py --data data.yaml --weights yolov5s.pt --batch 8 --name Model --img 640 --epochs 5

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias

In [ ]:
!python train.py --data data.yaml --weights yolov5s.pt --batch 8 --name Model --img 640 --epochs 15

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=15, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bia

In [ ]:
!python train.py --data data.yaml --weights yolov5s.pt --batch 8 --name Model --img 640 --epochs 25

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bia

In [ ]:
!python train.py --data data.yaml --weights yolov5l6.pt --batch 4 --name Model --img 1280 --epochs 25

train: weights=yolov5l6.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2

In [ ]:
!python train.py --data data.yaml --weights yolov5l6.pt --batch 4 --name Model --img 1280 --epochs 35

train: weights=yolov5l6.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=35, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2

In [ ]:
!python train.py --data data.yaml --weights yolov5x6.pt --batch 4 --name Model --img 1280 --epochs 5

train: weights=yolov5x6.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bi

In [ ]:
!python train.py --data data.yaml --weights yolov5x6.pt --batch 2 --name Model --img 1280 --epochs 15

train: weights=yolov5x6.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=15, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2

In [9]:
!python train.py --data data.yaml --weights yolov5x6.pt --batch 2 --name Model --img 1280 --epochs 25

train: weights=yolov5x6.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=2, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2

In [10]:
!python train.py --data data.yaml --weights yolov5s6.pt --batch 4 --name Model --img 1280 --epochs 5

train: weights=yolov5s6.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bi

In [11]:
!python train.py --data data.yaml --weights yolov5s6.pt --batch 4 --name Model --img 1280 --epochs 15

train: weights=yolov5s6.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=15, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
fatal: cannot change to '/content/drive/MyDrive/Polyp': No such file or directory
YOLOv5 🚀 2022-12-26 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_b

In [12]:
!python train.py --data data.yaml --weights yolov5s6.pt --batch 4 --name Model --img 1280 --epochs 25

Traceback (most recent call last):
  File "/usr/lib/python3.8/contextlib.py", line 75, in inner
    return func(*args, **kwds)
  File "/content/drive/MyDrive/Polyp Detection/yolov5/utils/general.py", line 350, in check_git_info
    import git
  File "/usr/local/lib/python3.8/dist-packages/git/__init__.py", line 44, in <module>
    from git.objects import *  # @NoMove @IgnorePep8
  File "/usr/local/lib/python3.8/dist-packages/git/objects/__init__.py", line 13, in <module>
    from .tag import *
  File "/usr/local/lib/python3.8/dist-packages/git/objects/tag.py", line 26, in <module>
    class TagObject(base.Object):
  File "/usr/local/lib/python3.8/dist-packages/git/objects/tag.py", line 46, in TagObject
    tagger: Union[None, "Actor"] = None,
  File "/usr/lib/python3.8/typing.py", line 258, in inner
    return cached(*args, **kwds)
  File "/usr/lib/python3.8/typing.py", line 358, in __getitem__
    parameters = tuple(_type_check(p, msg) for p in parameters)
  File "/usr/lib/python3.8/t